In [108]:
import torch.nn as nn

encoder_layer = nn.TransformerEncoderLayer(d_model=768, nhead=8)
model = nn.TransformerEncoder(encoder_layer, num_layers=2)

/home/mputo/miniconda3/envs/deeplearning/lib/python3.11/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [142]:
from transformers import AutoTokenizer, AutoModel
import torch

# load
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert = AutoModel.from_pretrained("bert-base-uncased")

# tokenize
text = ["hello world", "this is a sentence"]
batch = tokenizer(text, return_tensors="pt", padding=True)

# get embeddings
with torch.no_grad():
    x = bert.embeddings.word_embeddings(batch["input_ids"])  # [B, S, 768]


In [110]:
import torch.nn as nn

class TransformerWithClassifier(nn.Module):
    def __init__(self, encoder, embed_dim=256, num_classes=2):
        super().__init__()
        self.encoder = encoder
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, embedded_inputs, attention_mask=None):
        # Get pooled embedding from your transformer
        vec = self.encoder(embedded_inputs)   # [B, embed_dim]

        # Classify
        logits = self.classifier(vec)         # [B, num_classes]
        return logits


In [111]:
classifier = TransformerWithClassifier(model, embed_dim=768, num_classes=2)

torch.mean(classifier.forward(x), dim=1).shape

torch.Size([2, 2])

In [112]:
from datasets import load_dataset

en = load_dataset("allenai/c4", "en", streaming=True)

In [113]:
import random
from transformers import pipeline

generator = pipeline('text-generation', model='gpt2')

def get_text():
    for elem in en["train"]:
        if random.choice([0, 1]) == 0:
            yield [elem["text"], 0]
        else:
            words = random.randint(1, 10)
            length = min([len(elem["text"]), 64])
            yield [generator(elem["text"][:words], max_length=length)[0]['generated_text'], 1]


Device set to use cpu


In [180]:
criterion = nn.CrossEntropyLoss()
optimiezer = torch.optim.Adam(classifier.parameters(), lr=1e-4)

for text, classification in get_text():
    optimiezer.zero_grad()

    data = [text]
    batch = tokenizer(data, return_tensors="pt", padding=False)
    # get embeddings
    with torch.no_grad():
        x = bert.embeddings.word_embeddings(batch["input_ids"])  # [B, S, 768]

    logits = torch.mean(classifier.forward(x), dim=1) # [B, num_classes]

    loss = criterion(logits, torch.tensor(classification).unsqueeze(0))
    loss.backward()

    print(f"Loss: {loss.item()}")
    optimiezer.step()

    torch.save(classifier.state_dict(), 'model.pt')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Loss: 5.9344682693481445
Loss: 0.020762134343385696
Loss: 0.08047224581241608
Loss: 0.26034021377563477
Loss: 0.31861281394958496


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Loss: 0.3052065372467041


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Loss: 1.0279178619384766


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Loss: 3.4198310375213623


KeyboardInterrupt: 

In [181]:
def predict(text="BBQ classes in Missoula! Come and learn the art of grilling."):
    tokenizer.pad_token = tokenizer.eos_token
    classifier.load_state_dict(torch.load('model.pt'))

    data = [text]
    batch = tokenizer(data, return_tensors="pt", padding=False)
    # get embeddings
    with torch.no_grad():
        x = bert.embeddings.word_embeddings(batch["input_ids"])  # [B, S, 768]

    logits = torch.mean(classifier.forward(x), dim=1) # [B, num_classes]

    return logits

In [160]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel


model_name = "gpt2"
tokenizer2 = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval() 

def generate_next_token(text="Artificial Intelligence is transforming", max_length=64):
    encoded_input = tokenizer2(text, return_tensors='pt')
    input_ids = encoded_input['input_ids']

    with torch.no_grad():
        outputs = model(input_ids, max_length=max_length)

    logits = outputs.logits

    return logits


In [177]:
def get_words(logits, k=5):
    probabilities = nn.Softmax(dim=-1)(logits[0][-1])
    next_token_id = torch.topk(probabilities, k=k)

    probs = next_token_id.values

    return [tokenizer2.decode(next_token_id.indices, skip_special_tokens=True), probs]


In [182]:
text = "Artificial Intelligence is transforming"

logits = generate_next_token()
decoded, probs = get_words(logits)

print(decoded)
print(probs)

for item in decoded.split():
    generated = text + " " + item

    d = generator(generated, max_length=64)
    
    val = d[0]['generated_text']

    softmax = nn.Softmax()

    print(val)
    print(softmax(predict(val))[0][0].item())


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 the our itself into society
tensor([0.2434, 0.1072, 0.0431, 0.0290, 0.0254])


/home/mputo/miniconda3/envs/deeplearning/lib/python3.11/site-packages/torch/nn/modules/module.py:1775: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial Intelligence is transforming the workplace, cutting down on hours and allowing employers to take on less demand than they had before. It's one of Microsoft's greatest successes.

When Intel unveiled its $7 billion in Surface computers in 2007, the company focused on optimizing their performance with some real-world tasks. When
0.35757577419281006


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial Intelligence is transforming our lives in ways that few could have imagined. Its applications are beginning to show clear benefits to our lives—whether through the reduction of anxiety, stress, or our creativity. This new generation of computing technologies is rapidly becoming a reality.

But what may be more appealing to some is the
0.4748593866825104


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial Intelligence is transforming itself into something truly unique. It has an extraordinary ability to "discover" anything: our lives, whether it's we ourselves, or our entire world, or our children. Now, with its latest chip and chip-based machine, it's also looking to the future.

While
0.11951876431703568


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial Intelligence is transforming into a field of highly innovative, exciting research and engineering (IBE/E3). Its objective is to develop artificial intelligence techniques that could ultimately reduce the problems we face as a society today. By demonstrating the strength, agility, understanding, and resilience of a group of natural language processing algorithms using
0.5921380519866943
Artificial Intelligence is transforming society in ways we never imagined when we first proposed AI technologies.
0.4079658091068268


In [250]:
class Node:
    def __init__(self, parent, text, prob):
        self.state = text
        self.probability = prob
        self.N = 1
        self.children = []
        self.parent = parent

    def add_child(self, child_node):
        self.children.append(child_node)

    def get_children(self):
        return self.children

In [251]:
class Tree:
    def __init__(self, root):
        self.root = root

    def print_tree(self, node=None, level=0):
        if node is None:
            node = self.root
        print(' ' * level * 2 + f"{node.state} (Prob: {node.probability/node.N})")
        for child in node.get_children():
            self.print_tree(child, level + 1)

In [ ]:
tree = Tree(Node(None, "Artificial Intelligence is transforming the", 1.0))

In [266]:
import random

def pick_child(node):
    children = node.get_children()
    probabilities = [child.probability / child.N for child in children]
    total_prob = sum(probabilities).item()

    rand_val = random.uniform(0, total_prob)

    cumulative_probs = []
    current_sum = 0
    for prob in probabilities:
        current_sum += prob.item()
        cumulative_probs.append(current_sum)

    for i, cumulative in enumerate(cumulative_probs):
        if rand_val <= cumulative:
            return children[i]

In [267]:
rollouts = 10

for rollout in range(rollouts):
    node = tree.root

    while len(node.get_children()) > 0:
        node = pick_child(node)

    logits = generate_next_token(node.state)
    decoded, probs = get_words(logits)

    for i, next_word in enumerate(decoded.split()):
        print(node.state + " " + next_word)
        node.add_child(Node(node, node.state + " " + next_word, probs[i]))

    d = generator(text, max_length=64)

    softmax = nn.Softmax()

    probs = softmax(predict(val))[0][0].item()

    while node is not None:
        node.probability += probs
        node.N += 1
        node = node.parent

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial Intelligence is transforming the way
Artificial Intelligence is transforming the world
Artificial Intelligence is transforming the lives
Artificial Intelligence is transforming the field
Artificial Intelligence is transforming the future


/home/mputo/miniconda3/envs/deeplearning/lib/python3.11/site-packages/torch/nn/modules/module.py:1775: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial Intelligence is transforming the way we
Artificial Intelligence is transforming the way people
Artificial Intelligence is transforming the way you
Artificial Intelligence is transforming the way our
Artificial Intelligence is transforming the way the


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Artificial Intelligence is transforming the lives of
Artificial Intelligence is transforming the lives and,
Artificial Intelligence is transforming the lives we
Artificial Intelligence is transforming the lives in


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial Intelligence is transforming the way you work .,
Artificial Intelligence is transforming the way you work and
Artificial Intelligence is transforming the way you work in
Artificial Intelligence is transforming the way you work with


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial Intelligence is transforming the lives of millions
Artificial Intelligence is transforming the lives of people
Artificial Intelligence is transforming the lives of many
Artificial Intelligence is transforming the lives of our
Artificial Intelligence is transforming the lives of the


In [268]:
tree.print_tree()

Artificial Intelligence is transforming the (Prob: 0.34448398785157636)
  Artificial Intelligence is transforming the way (Prob: 0.26929500699043274)
    Artificial Intelligence is transforming the way we (Prob: 0.36109042167663574)
      Artificial Intelligence is transforming the way we think (Prob: 0.30042701959609985)
        Artificial Intelligence is transforming the way we think about (Prob: 0.7522686123847961)
        Artificial Intelligence is transforming the way we think and,. (Prob: 0.09218987822532654)
        Artificial Intelligence is transforming the way we think of (Prob: 0.06523390114307404)
      Artificial Intelligence is transforming the way we live (Prob: 0.08985525369644165)
      Artificial Intelligence is transforming the way we interact (Prob: 0.07239433377981186)
      Artificial Intelligence is transforming the way we work (Prob: 0.054217126220464706)
      Artificial Intelligence is transforming the way we do (Prob: 0.028207847848534584)
    Artificial Inte

In [275]:
max_child = tree.root
max_prob = -1

for child in tree.root.children:
    if child.probability / child.N > max_prob:
        max_prob = child.probability / child.N
        max_child = child
    print(child.state, child.probability / child.N)

print("--------------------")
print(max_child.state)

Artificial Intelligence is transforming the way tensor(0.2681)
Artificial Intelligence is transforming the world tensor(0.2650)
Artificial Intelligence is transforming the lives tensor(0.0209)
Artificial Intelligence is transforming the field tensor(0.0126)
Artificial Intelligence is transforming the future tensor(0.0114)
--------------------
Artificial Intelligence is transforming the way


In [294]:
def search(tree, rollout):
    for rollout in range(rollouts):
        node = tree.root

        while len(node.get_children()) > 0:
            node = pick_child(node)

        logits = generate_next_token(node.state)
        decoded, probs = get_words(logits)

        for i, next_word in enumerate(decoded.split()):
            node.add_child(Node(node, node.state + " " + next_word, probs[i]))

        val = generator(text, max_length=64)[0]['generated_text']

        softmax = nn.Softmax()

        probs = softmax(predict(val))[0][0].item()

        while node is not None:
            node.probability += probs
            node.N += 1
            node = node.parent

In [307]:
def find_max_child(node):
    max_child = node
    max_prob = -1

    for child in tree.root.children:
        if child.probability / child.N > max_prob:
            max_prob = child.probability / child.N
            max_child = child

    return max_child.state

In [310]:
tree = Tree(Node(None, "Happy customer", 1.0))

for i in range(10):
    search(tree, rollout=10)

    text = find_max_child(tree.root)
    tree = Tree(Node(None, text, 1.0))
    print(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Happy customer experience!


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Happy customer experience! I


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Happy customer experience! I was pleasantly surprised by the quality


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Happy customer experience! I was pleasantly surprised by the quality of


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Happy customer experience! I was pleasantly surprised by the quality of their


In [311]:
print(text)

Happy customer experience! I was pleasantly surprised by the quality of their


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# --- STEP 1: Load Llama 3.2 Model ---
# Use a public model name for Llama 3.2. Let's use the 3B Instruct version as an example.
model_id = "google/gemma-3-270m" 
tokenizer = AutoTokenizer.from_pretrained(model_id)
# Use 'device_map="auto"' to handle large models efficiently
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, # Recommended dtype for Llama 3 models
    device_map="auto"
)

# --- STEP 2: Prepare Input ---
prompt = "Tell me the steps to train a dog."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# --- STEP 3: Perform Forward Pass ---
with torch.no_grad():
    outputs = model(**inputs)

# --- STEP 4: Extract Logits ---
logits = outputs.logits # Shape: (batch_size, sequence_length, vocab_size)

# Logits for the NEXT token prediction (after the prompt)
next_token_logits = logits[0, -1, :] 

print(f"Llama 3.2 Logits Shape: {logits.shape}")
print(f"Logits for the first prediction token shape: {next_token_logits.shape}")


/home/mputo/.conda/envs/engine/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


Llama 3.2 Logits Shape: torch.Size([1, 10, 262144])
Logits for the first prediction token shape: torch.Size([262144])


In [2]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM

# Define the base Llama 3.2 model ID
MODEL_ID = "google/gemma-3-270m"

# --- Load the Tokenizer ---
# You must use the original tokenizer to maintain the same vocabulary and tokenization rules.
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# --- Load the Configuration ---
# The configuration object holds the model's architecture parameters (like layer count, hidden size, etc.).
config = AutoConfig.from_pretrained(MODEL_ID)

print(f"Original Model Layers: {config.num_hidden_layers}")

Original Model Layers: 18


In [3]:
NEW_LAYERS = 6
config.num_hidden_layers = NEW_LAYERS
small_llama_model = AutoModelForCausalLM.from_config(config)

In [4]:
print("-" * 50)
print(f"Vocabulary Size (must match original): {config.vocab_size}")
print(f"Hidden Dimension (must match original): {config.hidden_size}")
print(f"Number of layers in new model: {len(small_llama_model.model.layers)}")
print(f"Total model parameters: {small_llama_model.num_parameters():,}")

--------------------------------------------------
Vocabulary Size (must match original): 262144
Hidden Dimension (must match original): 640
Number of layers in new model: 6
Total model parameters: 201,214,592


In [5]:
device = torch.device("cuda")
small_llama_model.to(device)

Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 640, padding_idx=0)
    (layers): ModuleList(
      (0-5): 6 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear(in_features=640, out_features=1024, bias=False)
          (k_proj): Linear(in_features=640, out_features=256, bias=False)
          (v_proj): Linear(in_features=640, out_features=256, bias=False)
          (o_proj): Linear(in_features=1024, out_features=640, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear(in_features=640, out_features=2048, bias=False)
          (up_proj): Linear(in_features=640, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=640, bias=False)
          (act_fn): GELUTanh()
        )
        (input_layernorm): Gemma3RMSNorm((640,), eps=1e-06)
 

In [6]:
from datasets import load_dataset

en = load_dataset("allenai/c4", "en", streaming=True)

In [7]:
import torch
import torch.optim
import torch.nn.functional as F

kl_loss = torch.nn.KLDivLoss(reduction='batchmean')
optimizer = torch.optim.Adam(small_llama_model.parameters(), lr=1e-4)

for elem in en["train"]:
    optimizer.zero_grad()
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)
    
    outputs_student = small_llama_model(**inputs)

    loss = kl_loss(
        F.log_softmax(outputs_student.logits, dim=-1),
        F.softmax(outputs.logits, dim=-1)
    )
    print(loss.item())   

    loss.backward()
    optimizer.step()


88.5
77.5
70.5
65.5
61.75
58.75
56.5
54.5
53.0
51.75
50.5
49.5
48.5
47.75
47.0
46.25
45.5
44.75
44.25
43.5
42.75
42.25
41.5
41.0
40.25
39.5
39.0
38.25
37.5
37.0
36.25
35.5
34.75
34.25
33.5
32.75
32.0
31.375
30.75
30.125
29.375
28.625
28.0
27.375
26.75
26.0
25.375
24.625
24.0
23.375
22.75
22.0
21.375
20.75
20.125
19.625
19.0
18.375
17.75
17.125
16.625
16.125
15.5
15.0
14.5
14.0
13.5
13.0625
12.5625
12.0625
11.6875
11.25
10.875
10.4375
10.125
9.75
9.375
9.0
8.6875
8.375
8.125
7.84375
7.5625
7.3125
7.0625
6.8125
6.59375
6.375
6.1875
5.96875
5.8125
5.625
5.5
5.34375
5.1875
5.0625
4.90625
4.8125
4.6875
4.5625
4.4375
4.375
4.25
4.15625
4.0625
3.96875
3.921875
3.828125
3.75
3.671875
3.59375
3.546875
3.46875
3.390625
3.34375
3.296875
3.25
3.1875
3.125
3.09375
3.046875
2.984375
2.9375
2.90625
2.84375
2.8125
2.78125
2.765625
2.703125
2.65625
2.65625
2.609375
2.578125
2.53125
2.5
2.5
2.453125
2.4375
2.390625
2.359375
2.34375
2.328125
2.296875
2.265625
2.234375
2.234375
2.203125
2.1875
2.15625
2.1

KeyboardInterrupt: 

In [28]:
prompt = "Tell me the steps to get a"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# --- STEP 3: Perform Forward Pass ---
with torch.no_grad():
    outputs = small_llama_model(**inputs)

# --- STEP 4: Extract Logits ---
logits = outputs.logits # Shape: (batch_size, sequence_length, vocab_size)

# Logits for the NEXT token prediction (after the prompt)
next_token_logits = logits[0, -1, :] 

In [29]:
next_token_id = torch.argmax(next_token_logits, dim=-1)

In [30]:
tokenizer.decode(next_token_id)

' dog'